In [1]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import logit, loglogit, piecewiseFormula, nested
from biogeme.models import lognested
from biogeme.results import bioResults, pickle

In [2]:
df = pd.read_table("new_lpmc02.dat", index_col=0)

In [3]:
# create biogeme database

database = db.Database("LPMC",df)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

In [4]:
# Model

# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public = DefineVariable('cost_public_inc', cost_transit, database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving_inc = DefineVariable('cost_driving', (cost_driving_ccharge + cost_driving_fuel)*10, database)

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 *\
                car_ownership + Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving_inc + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1, 2: 1, 3: 1, 4: 1}


#Definitions of nests
N_SM = N_SM, [1, 2]
N_MOTOR = N_MOTOR, [3, 4]

nests = N_SM, N_MOTOR

In [5]:
# Model estimates


output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_nested_lpmc_sm_motor_driving_increased")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# delete previously saved html and pickle
for file in glob.glob(f"{filepath}*"):
    os.remove(file)
    
# The choice model is a logit, with availability conditions
logprob = lognested(V, av, nests, chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3660.096
rho bar square = 0.469
Output file: ./model-nested-output/logit_nested_lpmc_sm_motor_driving_increased.html


,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn_2,-0.188,0,0.138,-1.36,0.173,0.145,-1.29,0.195
CarOwn_3,-0.184,0,0.137,-1.34,0.18,0.157,-1.17,0.24
CarOwn_4,0.801,0,0.0902,8.87,0,0.0972,8.23,2.22e-16
Constant2,-2.18,0,0.44,-4.94,7.63e-07,0.403,-5.4,6.68e-08
Constant3,1.84,0,0.227,8.12,4.44e-16,0.217,8.5,0
Constant4,0.23,0,0.372,0.617,0.537,0.421,0.545,0.586
Cost,-0.00811,0,0.00194,-4.19,2.8e-05,0.00232,-3.5,0.000469
LAMBDA,-0.0564,0,0.102,-0.551,0.581,0.116,-0.487,0.626
N_MOTOR,1.77,0,0.324,5.46,4.65e-08,0.384,4.61,4.02e-06
N_SM,1,1,0.224,4.47,7.87e-06,0.307,3.26,0.0011


In [6]:
# market shares with increased costs

sum_weights = database.data['Weights'].sum()
S = database.getSampleSize()
sample_normalized_weight = Weights * S / sum_weights


prob_walking = nested(V, av, nests, 1)
prob_cycling = nested(V, av, nests, 2)
prob_public = nested(V, av, nests, 3)
prob_car = nested(V, av, nests, 4)


simulate = {'Prob. walking': prob_walking,
            'Prob. cycling': prob_cycling,
            'Prob. public': prob_public,
            'Prob. car': prob_car,
            'Weighted prob. walking': sample_normalized_weight * prob_walking,
            'Weighted prob. cycling': sample_normalized_weight * prob_cycling,
            'Weighted prob. public': sample_normalized_weight * prob_public,
            'Weighted prob. car': sample_normalized_weight * prob_car
           }


biogeme  = bio.BIOGEME(database, simulate)
#biogeme.modelName = filepath

betas = biogeme.freeBetaNames
results = bioResults(pickleFile=f"{filepath}.pickle")

beta_values = results.getBetaValues()

# get simulated values from prev saved beta values
simulated_values = biogeme.simulate(beta_values)

# calculate market shares 
marketShare_walking = 100 * simulated_values['Weighted prob. walking'].mean()
marketShare_cycling = 100 * simulated_values['Weighted prob. cycling'].mean()
marketShare_public = 100 * simulated_values['Weighted prob. public'].mean()
marketShare_car = 100 * simulated_values['Weighted prob. car'].mean()

# conf interval
b = results.getBetasForSensitivityAnalysis(betas, size=100)
left, right = biogeme.confidenceIntervals(b, 0.9)

# display results
lst_marketShares = [marketShare_walking, marketShare_cycling, marketShare_public, marketShare_car]
temp_names = ["marketShare_walking", "marketShare_cycling", "marketShare_public", "marketShare_car"]

print("Predicted market shares:\n")
for i in range(len(temp_names)):
    l = left[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    r = right[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    print(f"{temp_names[i]}: {lst_marketShares[i]:.2f}% ({l:.2f}%, {r:.2f}%)")

Predicted market shares:

marketShare_walking: 17.59% (15.16%, 20.12%)
marketShare_cycling: 2.84% (2.18%, 7.29%)
marketShare_public: 36.48% (33.38%, 41.10%)
marketShare_car: 43.09% (35.33%, 45.50%)


In [7]:
beta_values

{'CarOwn_2': -0.19824050646055802,
 'CarOwn_3': -0.23790293705451254,
 'CarOwn_4': 0.8195235809703245,
 'Constant2': -2.2939742642825074,
 'Constant3': 1.9821245195455324,
 'Constant4': 0.05733280198667734,
 'Cost': -0.08182360751779177,
 'LAMBDA': -0.016582907025691523,
 'N_MOTOR': 1.643091532516187,
 'N_SM': 1.0,
 'Time_Age_1': -0.05075525481281361,
 'Time_Age_2': -0.027886341475522256,
 'Time_Age_3': -0.033860197169865644,
 'Time_Age_4': -0.04203477494473885,
 'Total_TT1': -2.758329505351357,
 'Total_TT2': -1.272147785737909,
 'Total_TT3': -0.6262177999733233,
 'Total_TT4': -0.9575912174276116}